In [2]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\13. Section 3 Part 2 Consumption Expenditure Savings\2012_s3p2_f.csv"
file_2013=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\13. Section 3 Part 2 Consumption Expenditure Savings\2013_s3p2_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\13. Section 3 Part 2 Consumption Expenditure Savings\2014_s3p2_f.csv"

# Read excel files
df_2012 = pd.read_csv(file_2012)
df_2013 = pd.read_csv(file_2013)
df_2014 = pd.read_csv(file_2014)


In [3]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'pid': 'r_pid',
    's3p3q4': 's3p2_q4',
    's3p3q7': 's3p2_q5',
    's3p3q5': 'Save_Use',
    's3p3q6': 'Save_AccVal'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [4]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', None, None, 'r_pid', 's3p2_q4', 's3p2_q5', 'Save_Use', 'Save_AccVal'
]

mapping_2013 = [
    'hid', 'round', None, None, 'r_pid', 's3p2_q4', 's3p2_q5', None, None
]

mapping_2014 = [
    'hid', 'round', 's3p2_q1', 's3p2_q2', 'r_pid', 's3p2_q4', 's3p2_q5', None, None
]


In [5]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [6]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [7]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0.1', 'Unnamed: 0', 'hid', 'round', 'r_pid', 's3p2_q4', 'Save_Use', 'Save_AccVal', 's3p2_q5']
Appending data for column hid
Appending data for column round
Appending data for column r_pid
Appending data for column s3p2_q4
Appending data for column s3p2_q5
Appending data for column Save_Use
Appending data for column Save_AccVal
Processing DataFrame with columns: ['Unnamed: 0', 'hid', 'round', 'r_pid', 's3p2_q4', 's3p2_q5']
Appending data for column hid
Appending data for column round
Appending data for column r_pid
Appending data for column s3p2_q4
Appending data for column s3p2_q5
Processing DataFrame with columns: ['Unnamed: 0', 'hid', 'round', 's3p2_q1', 's3p2_q2', 'r_pid', 's3p2_q4', 's3p2_q5']
Appending data for column hid
Appending data for column round
Appending data for column s3p2_q1
Appending data for column s3p2_q2
Appending data for column r_pid
Appending data for column s3p2_q4
Appending data for column s3p2_q5


In [8]:
# Rename columns for the merged file (if needed)
rename_mapping = {
   'hid': 'HID',
    'round': 'Survey_Round',
    's3p2_q1': 'Save_Current',
    's3p2_q2': 'Save_12m',
    'r_pid': 'PID',
    's3p2_q4': 'Save_Where',
    's3p2_q5': 'Save_Tval'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('2. merged_CnE_Savings.csv', index=False)
